In [1]:
import tensorflow as tf
import os
import time
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds

In [2]:
data_dir = '/local-scratch/localhome/mkhademi/BOLD5000_2.0/'
batch_size = 20
dataset = tf.data.TFRecordDataset(filenames = [data_dir + 'image_data/bold5000_coco.tfrecords']).batch(batch_size)
# dataset = tf.data.TFRecordDataset(filenames = [data_dir + 'image_data/bold5000_imagenet.tfrecords']).shuffle(1916).batch(15)
train_ds = dataset.take(1800//batch_size)
test_ds = dataset.skip(1800//batch_size)
test_ds = test_ds.take(200//batch_size)

In [3]:
def tf_parse(eg):
    example = tf.io.parse_example(
        eg, {
            'x': tf.io.FixedLenFeature(shape=(71, 89, 72), dtype=tf.float32),
            'y': tf.io.FixedLenFeature(shape=(1000), dtype=tf.float32),
            'y_coco': tf.io.FixedLenFeature(shape=(90), dtype=tf.int64),
            'y_imagenet': tf.io.FixedLenFeature(shape=(1000), dtype=tf.int64),
            'face': tf.io.FixedLenFeature(shape=(1), dtype=tf.int64),
        })
    return example['x'], example['y_coco']
    
decoded = train_ds.map(tf_parse)
decoded_test = test_ds.map(tf_parse)

In [4]:
raw_example = next(iter(test_ds))
print(tf_parse(raw_example)[0].numpy().shape)
print(tf_parse(raw_example)[1].numpy().shape)
print(tf_parse(raw_example)[1].numpy().sum(axis=1))

(20, 71, 89, 72)
(20, 90)
[1 1 2 2 4 1 3 2 3 2 3 1 8 2 3 1 4 1 1 3]


In [5]:
for x_batch, y_batch in decoded_test.take(1):
    print(x_batch, y_batch)

tf.Tensor(
[[[[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  ...

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0.

In [6]:
model = tf.keras.Sequential([
    tf.keras.Input(shape=(71, 89, 72)),
    tf.keras.layers.Conv2D(4, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(2),
    tf.keras.layers.Conv2D(8, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(2),
    tf.keras.layers.Conv2D(16, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(2),
    tf.keras.layers.Flatten(),
    # tf.keras.layers.Dropout(rate=0.2),
    # tf.keras.layers.Dense(1000, activation='sigmoid'),
    # tf.keras.layers.Dense(100, activation='sigmoid'),
    tf.keras.layers.Dense(90, activation='sigmoid'),
    # tf.keras.layers.Softmax()
    ])
 
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 69, 87, 4)         2596      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 34, 43, 4)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 41, 8)         296       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 20, 8)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 18, 16)        1168      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 9, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1008)              0

In [7]:
model.compile(
    # loss='mse',
    # loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    loss = tf.keras.losses.BinaryCrossentropy(from_logits=False),
    # optimizer=keras.optimizers.RMSprop(),
    optimizer='adam',
    metrics=[tf.keras.metrics.MeanSquaredError()],
    # metrics=[tf.keras.metrics.CategoricalAccuracy()],
    # metrics=[tf.keras.metrics.CategoricalCrossentropy(from_logits=True)]
)
model.fit(decoded, epochs=40)

Epoch 1/40
90/90 [==============================] - 4s 24ms/step - loss: 8.2562 - mean_squared_error: 0.0726
Epoch 2/40
90/90 [==============================] - 2s 24ms/step - loss: 2.6873 - mean_squared_error: 0.0431
Epoch 3/40
90/90 [==============================] - 2s 24ms/step - loss: 1.3638 - mean_squared_error: 0.0412
Epoch 4/40
90/90 [==============================] - 2s 24ms/step - loss: 0.8799 - mean_squared_error: 0.0409
Epoch 5/40
90/90 [==============================] - 2s 24ms/step - loss: 0.6663 - mean_squared_error: 0.0390
Epoch 6/40
90/90 [==============================] - 2s 24ms/step - loss: 0.5739 - mean_squared_error: 0.0372
Epoch 7/40
90/90 [==============================] - 2s 25ms/step - loss: 0.5734 - mean_squared_error: 0.0347
Epoch 8/40
90/90 [==============================] - 2s 24ms/step - loss: 0.4616 - mean_squared_error: 0.0340
Epoch 9/40
90/90 [==============================] - 2s 25ms/step - loss: 0.3946 - mean_squared_error: 0.0326
Epoch 10/40
90/90 [

In [8]:
loss, accuracy = model.evaluate(decoded_test)

10/10 [==============================] - 4s 24ms/step - loss: 0.1599 - mean_squared_error: 0.0303


In [9]:
for x_batch, y_batch in decoded_test.take(1):
    print(model(x_batch))

tf.Tensor(
[[8.75878096e-01 1.49004636e-05 1.24047175e-01 ... 1.36710325e-04
  6.35057688e-07 1.75762549e-03]
 [4.91979808e-01 6.54029122e-07 8.32816679e-03 ... 3.96131771e-03
  2.76187592e-07 6.44135298e-06]
 [7.28947401e-01 1.32652244e-03 3.55935208e-02 ... 7.49320956e-03
  8.64914091e-06 1.54311128e-03]
 ...
 [3.43875229e-01 8.26416246e-04 7.58403987e-02 ... 9.39124380e-04
  2.74335505e-08 1.02941692e-03]
 [7.22711623e-01 1.01048296e-04 6.31126855e-03 ... 7.43262179e-04
  5.37258904e-09 3.27462505e-04]
 [2.88291961e-01 6.91575697e-05 6.25326812e-01 ... 1.83559168e-04
  3.51817448e-06 1.80750922e-05]], shape=(20, 90), dtype=float32)
